In [3]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import math
import random
sns.set_theme(style="whitegrid")
os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [2]:
base = "results/motif_analysis/meme/fimo/*"
target = "GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1"

In [42]:
for sample in glob.glob(base):
    summary = sample + "/summarize_results/motif_summary.txt"
    sample_name = summary.split("/")[4]
    if sample_name == target:
        with open(summary) as s:
            info = []
            for line in s:
                info.append(line.strip().split("\t"))
                #for i in range(0, 15): 
                    #info.append(line.strip().split("\t")[i])
            df = pd.DataFrame(info)

In [43]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,loop_1_anchor_1,chr1,770000,780000,chr1,778906,778921,MA1937.1,ERF::HOXB13,-,16.1552,9.19e-07,1,CCGGGAAGTCGTAAAG
1,loop_1_anchor_1,chr1,770000,780000,chr1,778906,778922,MA1958.1,HOXD12::ELK1,-,19.8182,1.52e-07,1,ACCGGGAAGTCGTAAAG
2,loop_1_anchor_1,chr1,770000,780000,chr1,778985,778997,MA0499.2,MYOD1,+,17.0081,1.52e-07,0.238,CGGCACCTGTCGC
3,loop_1_anchor_1,chr1,770000,780000,chr1,778985,778997,MA0745.2,SNAI2,+,16.4388,6.57e-07,0.158,CGGCACCTGTCGC
4,loop_1_anchor_1,chr1,770000,780000,chr1,779372,779387,MA1596.1,ZNF460,-,17.9888,3.46e-07,0.00163,GCCTCGACCTCCCAAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168233,loop_89206_anchor_2,chrX,155610000,155620000,chrX,155612913,155612924,MA1653.1,ZNF148,+,19.6327,6.76e-08,0.00787,CTCCCCTCCCCC
168234,loop_89206_anchor_2,chrX,155610000,155620000,chrX,155612915,155612924,MA1965.1,SP5,+,15.898,4.13e-07,0.0161,CCCCTCCCCC
168235,loop_89206_anchor_2,chrX,155610000,155620000,chrX,155612956,155612965,MA0599.1,KLF5,+,17,3.39e-07,0.0156,GCCCCGCCCC
168236,loop_89206_anchor_2,chrX,155610000,155620000,chrX,155612957,155612974,MA1981.1,ZNF530,-,19.2143,2.3e-07,0.0317,GGAAGGAGTGGGGCGGGG


In [52]:
path = "results/motif_analysis/meme/fimo/GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1/summarize_results/motif_summary.txt"
loops = {}
with open(path) as f:
    for line in f:
        loop = line.strip().split("\t")[0].split("_a")[0]
        anchor = line.strip().split("\t")[0].split("_")[3]
        if loop not in loops:
            loops[loop] = []
        if anchor not in loops.get(loop):
            loops.get(loop).append(anchor)
print(len(loops))
count1 = 0
count2 = 0
for l in loops:
    if (len(loops[l]) == 1):
        count1 = count1 + 1
    if (len(loops[l]) == 2):
        count2 = count2 + 1

print(count1)
print(count2)

18387
16863
1524


In [44]:
def input_num_unique_loops(sample):
    path = sample + "/anchors.txt"
    with open(path) as a:
        loops = {}
        for line in a:
                loop = line.strip().split("\t")[3].split("_a")[0]
                anchor = line.strip().split("\t")[3].split("_")[3]
                if loop not in loops:
                    loops[loop] = []
                if anchor not in loops.get(loop):
                    loops.get(loop).append(anchor)
        return (len(loops), sum((len(array) for array in loops.values())))      

In [45]:
def output_num_unique_loops(sample):
    summary = sample + "/summarize_results/summary.txt"
    with open(summary) as s:
        loops = {}
        for line in s:
            loop = line.strip().split("\t")[3].split("_a")[0]
            anchor = line.strip().split("\t")[3].split("_")[3]
            if loop not in loops:
                loops[loop] = []
            if anchor not in loops.get(loop):
                loops.get(loop).append(anchor)
    return (len(loops), sum((len(array) for array in loops.values())), sum((1 for array in loops.values() if len(array) == 2)))

In [46]:
def output_num_peaks(sample):
    summary = sample + "/summarize_results/summary.txt"
    with open(summary) as s:
        peaks = set()
        for line in s:
            curr = line.strip().split("\t")
            peak = []
            peak.append(curr[0])
            peak.append(curr[1])
            peak.append(curr[2])
            peak_string = " ".join(peak)
            peaks.add(peak_string)
    return len(peaks)

In [47]:
def num_peaks_with_multiple_loop_overlaps(sample):
    summary = sample + "/summarize_results/peaks.txt"
    with open(summary) as s:
        peaks = {}
        for line in s:
            curr = line.strip().split("\t")
            peak = []
            peak.append(curr[0])
            peak.append(curr[1])
            peak.append(curr[2])
            peak_string = " ".join(peak)
            if peak_string not in peaks: 
                peaks[peak_string] = set()
            peaks[peak_string].add(curr[3])
        max_degree = 0
        for peak in peaks.keys():
            if len(peaks.get(peak)) > max_degree:
                max_degree = len(peaks.get(peak))
    return (len(peaks), sum((1 for st in peaks.values() if len(st) > 1)), max_degree)

In [39]:
paths = glob.glob("results/motif_analysis/meme/fimo/*")
paths.sort()
for sample in paths:
    sample_name = sample.split("/")[4]
    print(sample_name)
    input_data = input_num_unique_loops(sample)
    output_data = output_num_unique_loops(sample)
    print("Input Unique Loops:", input_data[0])
    print("Input Unique Anchors:", input_data[1])
    print("Input Best Peaks Overlapping Anchors (Unique b/c one peak may overlap multiple anchors):", num_peaks_with_multiple_loop_overlaps(sample)[0])
    print("Peaks with Multiple Overlapping Loop Anchors:", num_peaks_with_multiple_loop_overlaps(sample)[1])
    print("Max Degree:", num_peaks_with_multiple_loop_overlaps(sample)[2])
    print("Loops with Motif Overlap:", output_data[0])
    print("Anchors with Motif Overlap:", output_data[1])
    print("Loops with Motifs in Both Anchors:", output_data[2])
    print("Peaks with Motif Overlap (Unique):", output_num_peaks(sample))
    print()
    
# for each anchor, number of loops that use that anchor

GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1
Input Unique Loops: 42043
Input Unique Anchors: 47671
Input Best Peaks Overlapping Anchors (Unique): 23372
Peaks with Multiple Overlapping Loop Anchors: 1940
Max Degree: 3
Loops with Motif Overlap: 19229
Anchors with Motif Overlap: 20858
Loops with Motifs in Both Anchors: 1629
Peaks with Motif Overlap (Unique): 18979

GM12878.GSE101498.Homo_Sapiens.H3K27ac.b2
Input Unique Loops: 38087
Input Unique Anchors: 43627
Input Best Peaks Overlapping Anchors (Unique): 22491
Peaks with Multiple Overlapping Loop Anchors: 1853
Max Degree: 3
Loops with Motif Overlap: 18364
Anchors with Motif Overlap: 20140
Loops with Motifs in Both Anchors: 1776
Peaks with Motif Overlap (Unique): 18342

GM12878.GSE115524.Homo_Sapiens.CTCF.b1
Input Unique Loops: 36420
Input Unique Anchors: 42094
Input Best Peaks Overlapping Anchors (Unique): 26691
Peaks with Multiple Overlapping Loop Anchors: 286
Max Degree: 2
Loops with Motif Overlap: 16511
Anchors with Motif Overlap: 18329


In [141]:
def input_num_unique_anchors(sample):
    input_anchors = sample + "/anchors.txt"
    output_anchors = sample + "/summarize_results/summary.txt"
    with open(input_anchors) as i, open(output_anchors) as o:
        input_anchors = []
        output_anchors = []
        for line in i:
            input_anchors.append(line.strip().split("\t")[3])
        for line in o:
            output_anchors.append(line.strip().split("\t")[3])
    return (list(set(input_anchors)), list(set(output_anchors)))   

In [ ]:
def peak_bins(sample):
    summary = sample + "/summarize_results/summary.txt"
    with open(summary) as s:
        peaks = {}
        for line in s:
            curr = line.strip().split("\t")
            peak = []
            peak.append(curr[0])
            peak.append(curr[1])
            peak.append(curr[2])
            peak_string = " ".join(peak)
            if peak_string not in peaks: 
                peaks[peak_string] = set()
            peaks[peak_string].add(curr[3])
        max_degree = 0
        for peak in peaks.keys():
            if len(peaks.get(peak)) > max_degree:
                max_degree = len(peaks.get(peak))
    return (len(peaks), sum((1 for st in peaks.values() if len(st) > 1)), max_degree)

In [146]:
def suffle_anchors(sample):
    info = input_num_unique_anchors(sample)
    print(len(info[0]))
    print(len(info[1]))
    shuffled = random.sample(info[0], len(info[1]))
    print(len(shuffled))
    print(shuffled)

In [151]:
suffle_anchors("results/motif_analysis/meme/fimo/GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1")

47671
20858
20858
['loop_48400_anchor_1', 'loop_61274_anchor_2', 'loop_11436_anchor_1', 'loop_23454_anchor_1', 'loop_30186_anchor_2', 'loop_62761_anchor_1', 'loop_10335_anchor_2', 'loop_75819_anchor_1', 'loop_60580_anchor_2', 'loop_43353_anchor_1', 'loop_56920_anchor_1', 'loop_53642_anchor_2', 'loop_25597_anchor_1', 'loop_19249_anchor_2', 'loop_40119_anchor_2', 'loop_26849_anchor_2', 'loop_79857_anchor_2', 'loop_66994_anchor_2', 'loop_62809_anchor_2', 'loop_9703_anchor_1', 'loop_64011_anchor_1', 'loop_16811_anchor_2', 'loop_12703_anchor_2', 'loop_59573_anchor_2', 'loop_60682_anchor_2', 'loop_25692_anchor_2', 'loop_27746_anchor_2', 'loop_13329_anchor_2', 'loop_5505_anchor_1', 'loop_67379_anchor_2', 'loop_32745_anchor_1', 'loop_80888_anchor_1', 'loop_80421_anchor_2', 'loop_85789_anchor_2', 'loop_12083_anchor_2', 'loop_65617_anchor_1', 'loop_13113_anchor_2', 'loop_45708_anchor_2', 'loop_49834_anchor_1', 'loop_44134_anchor_2', 'loop_74000_anchor_1', 'loop_52826_anchor_2', 'loop_55702_ancho

In [122]:
my_list = [1, 2, 3, 4, 5]
my_dict = {'hi' : 1, 'hello' : 2, "bye": 3, "cool" : 4, "no" : 5}
print(len())
random.sample(list(my_dict.keys()), 3)

['hi', 'hello', 'bye', 'cool', 'no']


['bye', 'no', 'hello']

## Summarize FIMO Output

In [129]:
sample = "GM12878.GSE115524.Homo_Sapiens.CTCF.b2"
file = "results/motif_analysis/meme/fimo/" + sample + "/summarize_results_050223/loops_overlap_motifs.sorted.uniq.txt"
df = pd.read_csv(file, sep = "\t", header=None)

In [130]:
columns = ["chr1", "start1", "end1", "chr2", "start2", "end2", "motif_chr", "motif_start", "motif_end", "motif_ID", "motif_name"]
df.columns = columns

In [131]:
df

,chr1,start1,end1,chr2,start2,end2,motif_chr,motif_start,motif_end,motif_ID,motif_name
0,chr1,770000,780000,chr1,820000,830000,chr1,778906,778921,MA1937.1,ERF::HOXB13
1,chr1,770000,780000,chr1,820000,830000,chr1,778906,778922,MA1958.1,HOXD12::ELK1
2,chr1,770000,780000,chr1,820000,830000,chr1,778985,778997,MA0499.2,MYOD1
3,chr1,770000,780000,chr1,820000,830000,chr1,778985,778997,MA0745.2,SNAI2
4,chr1,900000,910000,chr1,930000,940000,chr1,904751,904785,MA1930.1,CTCF
...,...,...,...,...,...,...,...,...,...,...,...
54345,chrX,154980000,154990000,chrX,155040000,155050000,chrX,154980986,154981019,MA1929.1,CTCF
54346,chrX,154980000,154990000,chrX,155040000,155050000,chrX,155049191,155049209,MA0139.1,CTCF
54347,chrX,155090000,155100000,chrX,155130000,155140000,chrX,155094661,155094679,MA0139.1,CTCF
54348,chrX,155090000,155100000,chrX,155130000,155140000,chrX,155137627,155137650,MA1969.1,THRA


In [132]:
df_1 = df.copy()
df_1['loop'] = df_1.apply(lambda x: str(x["chr1"]) + "/" + str(x["start1"]) + "/" +  str(x["end1"]) + "/" + str(x["chr2"]) + "/" + str(x["start2"]) + "/" + str(x["end2"]), axis=1)
df_1['anchor'] = df_1.apply(lambda x: "1" if x["motif_start"] >= x["start1"] and x["motif_start"] <= x["end1"] else "2", axis=1)
df_1['anchor_1'] = df_1.apply(lambda x: str(x["chr1"]) + "/" + str(x["start1"]) + "/" + str(x["end1"]), axis=1)
df_1['anchor_2'] = df_1.apply(lambda x: str(x["chr2"]) + "/" + str(x["start2"]) + "/" + str(x["end2"]), axis=1)

In [133]:
#df_1['anchor'] = df_1.apply(lambda x: "1" if x["motif_start"] >= x["start1"] and x["motif_start"] <= x["end1"] else "2", axis=1)
df_1['motif_ID_1'] = df_1.apply(lambda x: x["motif_ID"] if x["anchor"] == "1" else "", axis=1)
df_1['motif_name_1'] = df_1.apply(lambda x: x["motif_name"] if x["anchor"] == "1" else "", axis=1)
df_1['motif_ID_2'] = df_1.apply(lambda x: x["motif_ID"] if x["anchor"] == "2" else "", axis=1)
df_1['motif_name_2'] = df_1.apply(lambda x: x["motif_name"] if x["anchor"] == "2" else "", axis=1)

In [134]:
df_1

,chr1,start1,end1,chr2,start2,end2,motif_chr,motif_start,motif_end,motif_ID,motif_name,loop,anchor,anchor_1,anchor_2,motif_ID_1,motif_name_1,motif_ID_2,motif_name_2
0,chr1,770000,780000,chr1,820000,830000,chr1,778906,778921,MA1937.1,ERF::HOXB13,chr1/770000/780000/chr1/820000/830000,1,chr1/770000/780000,chr1/820000/830000,MA1937.1,ERF::HOXB13,,
1,chr1,770000,780000,chr1,820000,830000,chr1,778906,778922,MA1958.1,HOXD12::ELK1,chr1/770000/780000/chr1/820000/830000,1,chr1/770000/780000,chr1/820000/830000,MA1958.1,HOXD12::ELK1,,
2,chr1,770000,780000,chr1,820000,830000,chr1,778985,778997,MA0499.2,MYOD1,chr1/770000/780000/chr1/820000/830000,1,chr1/770000/780000,chr1/820000/830000,MA0499.2,MYOD1,,
3,chr1,770000,780000,chr1,820000,830000,chr1,778985,778997,MA0745.2,SNAI2,chr1/770000/780000/chr1/820000/830000,1,chr1/770000/780000,chr1/820000/830000,MA0745.2,SNAI2,,
4,chr1,900000,910000,chr1,930000,940000,chr1,904751,904785,MA1930.1,CTCF,chr1/900000/910000/chr1/930000/940000,1,chr1/900000/910000,chr1/930000/940000,MA1930.1,CTCF,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54345,chrX,154980000,154990000,chrX,155040000,155050000,chrX,154980986,154981019,MA1929.1,CTCF,chrX/154980000/154990000/chrX/155040000/155050000,1,chrX/154980000/154990000,chrX/155040000/155050000,MA1929.1,CTCF,,
54346,chrX,154980000,154990000,chrX,155040000,155050000,chrX,155049191,155049209,MA0139.1,CTCF,chrX/154980000/154990000/chrX/155040000/155050000,2,chrX/154980000/154990000,chrX/155040000/155050000,,,MA0139.1,CTCF
54347,chrX,155090000,155100000,chrX,155130000,155140000,chrX,155094661,155094679,MA0139.1,CTCF,chrX/155090000/155100000/chrX/155130000/155140000,1,chrX/155090000/155100000,chrX/155130000/155140000,MA0139.1,CTCF,,
54348,chrX,155090000,155100000,chrX,155130000,155140000,chrX,155137627,155137650,MA1969.1,THRA,chrX/155090000/155100000/chrX/155130000/155140000,2,chrX/155090000/155100000,chrX/155130000/155140000,,,MA1969.1,THRA


In [135]:
## drop rows for which at least one anchor is not conserved
# c="results/motif_analysis/conserved_anchors/meme_out/conserved_anchors.sorted.bed"
# c_df = pd.read_csv(c, header=None, sep="\t")
# conserved = []
# for i, sr in c_df.iterrows():
#     conserved.append(sr[0] + "/" + str(sr[1]) + "/" + str(sr[2]))
    
# df_1["status"] = df_1.apply(lambda x: "conserved" if x["anchor_1"] in conserved or x["anchor_2"] in conserved else "drop", axis=1)

In [136]:
# df_1.loc[df_1["status"] == "drop"]

In [137]:
df_1 = df_1.drop(columns=["motif_chr", "motif_start", "motif_end", "motif_ID", "motif_name", "anchor", "anchor_1", "anchor_2"])

In [138]:
df_1 = df_1.groupby(["loop"]).agg({"chr1" : "first", "start1" : "first", "end1" : "first", "chr2" : "first", "start2" : "first", "end2" : "first", "motif_ID_1" : pd.Series.unique, "motif_name_1" : pd.Series.unique, "motif_ID_2" : pd.Series.unique, "motif_name_2" : pd.Series.unique})

In [139]:
df_1 = df_1.sort_values(by=["chr1", "start1", "start2"])

In [140]:
df_1['motif_ID_1'] = df_1.apply(lambda x: ",".join(x["motif_ID_1"]), axis=1)
df_1['motif_ID_1'] = df_1.apply(lambda x: x["motif_ID_1"].strip(","), axis=1)
df_1['motif_ID_1'] = df_1.apply(lambda x: "None" if len(x["motif_ID_1"]) == 0 else x["motif_ID_1"], axis=1)

df_1['motif_name_1'] = df_1.apply(lambda x: ",".join(x["motif_name_1"]), axis=1)
df_1['motif_name_1'] = df_1.apply(lambda x: x["motif_name_1"].strip(","), axis=1)
df_1['motif_name_1'] = df_1.apply(lambda x: "None" if len(x["motif_name_1"]) == 0 else x["motif_name_1"], axis=1)

df_1['motif_ID_2'] = df_1.apply(lambda x: ",".join(x["motif_ID_2"]), axis=1)
df_1['motif_ID_2'] = df_1.apply(lambda x: x["motif_ID_2"].strip(","), axis=1)
df_1['motif_ID_2'] = df_1.apply(lambda x: "None" if len(x["motif_ID_2"]) == 0 else x["motif_ID_2"], axis=1)

df_1['motif_name_2'] = df_1.apply(lambda x: ",".join(x["motif_name_2"]), axis=1)
df_1['motif_name_2'] = df_1.apply(lambda x: x["motif_name_2"].strip(","), axis=1)
df_1['motif_name_2'] = df_1.apply(lambda x: "None" if len(x["motif_name_2"]) == 0 else x["motif_name_2"], axis=1)

In [141]:
df_1 = df_1.reset_index().drop(columns=["loop"])

In [142]:
df_1

,chr1,start1,end1,chr2,start2,end2,motif_ID_1,motif_name_1,motif_ID_2,motif_name_2
0,chr1,770000,780000,chr1,820000,830000,"MA1937.1,MA1958.1,MA0499.2,MA0745.2","ERF::HOXB13,HOXD12::ELK1,MYOD1,SNAI2",None,None
1,chr1,900000,910000,chr1,930000,940000,"MA1930.1,MA0139.1,MA1102.2,MA1596.1","CTCF,CTCFL,ZNF460","MA0139.1,MA1930.1",CTCF
2,chr1,970000,980000,chr1,1300000,1310000,"MA1930.1,MA0103.3,MA0522.3,MA1648.1","CTCF,ZEB1,TCF3,TCF12",None,None
3,chr1,980000,990000,chr1,1000000,1010000,"MA1719.1,MA1929.1,MA0139.1,MA0155.1","ZNF816,CTCF,INSM1","MA0471.2,MA0139.1","E2F6,CTCF"
4,chr1,1010000,1020000,chr1,1060000,1070000,"MA0646.1,MA0697.1,MA0751.1,MA1584.1,MA0696.1","GCM1,ZIC3,ZIC4,ZIC5,ZIC1","MA1976.1,MA1929.1,MA0139.1,MA1930.1,MA0039.4","ZNF320,CTCF,KLF4"
...,...,...,...,...,...,...,...,...,...,...
19109,chrX,154400000,154410000,chrX,154520000,154530000,None,None,MA0599.1,KLF5
19110,chrX,154420000,154430000,chrX,154520000,154530000,None,None,MA0599.1,KLF5
19111,chrX,154910000,154920000,chrX,155140000,155150000,None,None,"MA1102.2,MA0696.1","CTCFL,ZIC1"
19112,chrX,154980000,154990000,chrX,155040000,155050000,"MA0050.2,MA1929.1","IRF1,CTCF",MA0139.1,CTCF


In [143]:
df_1.to_csv("results/motif_analysis/meme/fimo/" + sample + "/summarize_results_050223/summary.txt", sep="\t", header=True, index=False)